Automação Web e Busca de Informações com Python
Desafio:
Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:

Dólar
Euro
Ouro
Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

Usaremos o selenium
Importante: baixar o webdriver

!pip install selenium

In [6]:
#1 Importar webdriver (Google Chrome -> chromedriver ou FireFox -> geckodriver)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#Navegador em 2° após validação de codigo
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True 
# navegador = webdriver.Chrome(options=chrome_options)

navegador = webdriver.Chrome()

#1.1 Pegar cotação do Dolar 
#1.2 Euro
#1.3 Ouro

navegador.get("https://www.google.com/")
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys("cotação dolar")
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(
                       'xpath',
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                       ).get_attribute('data-value')
navegador.get("https://www.google.com/")

navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys("cotação euro")
navegador.find_element('xpath','//*[@id="APjFqb"]').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(
                       'xpath',
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                       ).get_attribute('data-value')
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(
                       'xpath',
                       '//*[@id="comercial"]'
                       ).get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)
print(cotacao_euro)
print(cotacao_dolar)
navegador.quit()

298.21
5.241499999999999
4.7692


In [7]:
#2 Importar a base de dados(produtos) e atualizar a base

import pandas as pd

produtos = pd.read_excel(r"C:\Users\glmarcos\Downloads\Produtos.xlsx")

#print(produtos.info())
display(produtos)
print(cotacao_ouro)
print(cotacao_euro)
print(cotacao_dolar)

#print(produtos["Preço Original"])
#prod_dolar = produtos["Preço Original"]*cotacao_dolar
#print(prod_dolar)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


298.21
5.241499999999999
4.7692


In [8]:
# 3 Atualizar cotação
produtos.loc[produtos["Moeda"] == "Dólar","Cotação"] = float(cotacao_dolar)
produtos.loc[produtos["Moeda"] == "Euro","Cotação"] = float(cotacao_euro)
produtos.loc[produtos["Moeda"] == "Ouro","Cotação"] = float(cotacao_ouro)

# 3.1 Recalcular os preços
# 3.2 Preço de compra = cotação * preço original
produtos["Preço de Compra"] = produtos["Cotação"] * produtos["Preço Original"] 

# 3.3 Preço de venda = preço de compra * margem
produtos["Preço de Venda"] = produtos["Preço de Compra"] * produtos["Margem"]
#produtos["Preço de Venda"] = produtos["Preço de Venda"].map("R${:.2f}".format)
#produtos["Preço de Compra"] = produtos["Preço de Compra"].map("R${:.2f}".format)

display(produtos)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.7692,4769.152308,1.40,6676.813231
1,Carro Renault,4500.00,Euro,5.2415,23586.750000,2.00,47173.500000
2,Notebook Dell,899.99,Dólar,4.7692,4292.232308,1.70,7296.794924
3,IPhone,799.00,Dólar,4.7692,3810.590800,1.70,6478.004360
4,Carro Fiat,3000.00,Euro,5.2415,15724.500000,1.90,29876.550000
5,Celular Xiaomi,480.48,Dólar,4.7692,2291.505216,2.00,4583.010432
6,Joia 20g,20.00,Ouro,298.2100,5964.200000,1.15,6858.830000


In [5]:
#4 Exportar base atualizada
produtos.to_excel("Produtos_universo.xlsx",index = False)